In [1]:
import pandas as pd
from typing import List

### Reading the Data and Initial Stats

In [2]:
movies_df: pd.DataFrame = pd.read_csv('input/all_movies.csv')
movies_df.sample(20)

,movieId,title,genres
41298,156235,The Escape (1964),Drama
29361,127587,The Seven Males (1951),Comedy|western
41033,155589,Noin 7 veljestä (1968),(no genres listed)
40929,155314,Between Eleven and Midnight (1949),(no genres listed)
6511,6620,American Splendor (2003),Comedy|drama
3470,3560,Empire of Passion (a.k.a. In the Realm of Pass...,Crime|drama|ROMANCE
22583,107213,"Truth About Emanuel, The (2013)",Drama|thriller
26672,120857,Bloodsuckers (2005),Action|horror|SCI-FI
29330,127341,Longshot (2001),Action|comedy|CRIME|drama|Romance|THRILLER
52391,180687,I'll See You in My Dreams (1951),Comedy|drama


In [3]:
movies_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|animation|CHILDREN|comedy|Fantasy
1,2,Jumanji (1995),Adventure|children|FANTASY
2,3,Grumpier Old Men (1995),Comedy|romance
3,4,Waiting to Exhale (1995),Comedy|drama|ROMANCE
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
58093,193876,The Great Glinka (1946),(no genres listed)
58094,193878,Les tribulations d'une caissière (2011),Comedy
58095,193880,Her Name Was Mumu (2016),Drama
58096,193882,Flora (2017),Adventure|drama|HORROR|sci-fi


In [4]:
movies_df.shape

(58098, 3)

In [5]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58098 entries, 0 to 58097
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  58098 non-null  int64 
 1   title    58098 non-null  object
 2   genres   58098 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.3+ MB


## Step 1: Cleaning
### You shouldn't necessarily assume that your data is good.  It could be very sparse and not have much there. There could be duplication, poorly recorded or empty values, or with large text there could be a lot of garbage in there if it was an open text field

In [ ]:

movies_cleaned_df = movies_df.copy()
movies_cleaned_df['genres'] = movies_cleaned_df['genres'].apply(lower_case_and_strip_spaces_notebook)

In [6]:
def lower_case_and_strip_spaces_notebook(input: str) -> str:
    return None

In [7]:
input_1 = " comedy|FANTASY|Action "
expected_1 = "comedy|fantasy|action"

input_2 = " FANTASY "
expected_2 = "fantasy"

input_3 = "comedy"
expected_3 = "comedy"

In [8]:
def test_lower_case_and_strip_spaces(input: str, expected: str):
    actual = lower_case_and_strip_spaces_notebook(input)
    assert actual == expected

In [9]:
test_lower_case_and_strip_spaces(input_1, expected_1)

AssertionError: 

In [10]:
test_lower_case_and_strip_spaces(input_2, expected_2)

AssertionError: 

In [ ]:
test_lower_case_and_strip_spaces(input_3, expected_3)

In [ ]:
movies_cleaned_df.sample(20)

In [ ]:
movies_cleaned_df = movies_cleaned_df.loc[movies_cleaned_df['genres'] != '(no genres listed)']

In [ ]:
movies_cleaned_df.shape

In [ ]:
assert movies_cleaned_df.shape[0] < movies_df.shape[0]

In [ ]:
movies_cleaned_df.head(5)

## Checking for Duplicates
### We need to define what a "duplicate" is

In [ ]:
duplicated_by_title = movies_cleaned_df.duplicated(['title'])

In [ ]:
duplicated_by_title_df = movies_cleaned_df.loc[duplicated_by_title]
duplicated_by_title_df

In [ ]:
duplicated_by_title_and_genre = movies_cleaned_df.duplicated(['title', 'genres'])

In [ ]:
duplicated_by_title_and_genre_df = movies_cleaned_df.loc[duplicated_by_title_and_genre]
duplicated_by_title_and_genre_df

In [ ]:
newdf=pd.concat([duplicated_by_title_and_genre_df,duplicated_by_title_df]).drop_duplicates(keep=False)
records_duplicated_by_title_only = set(newdf['title'])
records_duplicated_by_title_only

### Now we can locate an example using the titles in our list.

In [ ]:
ALADDIN = 'Aladdin (1992)'

In [ ]:
def get_aladdin_example(df: pd.DataFrame) -> pd.DataFrame:
    return df.loc[df['title'] == ALADDIN]

In [ ]:
movies_cleaned_df.loc[movies_cleaned_df['title'] == ALADDIN]

## Step 2: Feature Preparation

What is a feature?

A descriptive attribute that can be used in our algorithms.

Examples:
    - If we are trying to predict house prices, square footage could be a feature we use to predict the house price
    - In our case, as we try to find movies similar to our movie the "feature" we will be focusing on is the "genres" description
    - We need to "prepare" the columns data in such a way that we can compare one genre description to another and get some measure of similarity

### Let's prepare our genres list. First we need to group by movie title.

In [ ]:
movies_grouped_by_title_df = movies_cleaned_df.copy()
movies_grouped_by_title_df = movies_grouped_by_title_df.groupby('title').agg({'genres': lambda x: x.to_list()}).reset_index()

In [ ]:
movies_grouped_by_title_df

In [ ]:
get_aladdin_example(movies_grouped_by_title_df)

Clean up the genres lists.

In [ ]:
from analysis.utils.cleaning import combine_genres_list

movies_unique_genres_df = movies_grouped_by_title_df.copy()
movies_unique_genres_df['genres'] = movies_unique_genres_df['genres'].apply(combine_genres_list)

In [ ]:
movies_unique_genres_df.sample(20)

In [ ]:
get_aladdin_example(movies_unique_genres_df)

### Let's think about our recommendation engine now again.  Let's say that we want to recommend movies to by recommending the movies with the most similar genres list.

### Collect Unique List of Genres

In [ ]:
for_genres_list_df = movies_unique_genres_df.copy()
for_genres_list_df = for_genres_list_df['genres'].explode().reset_index()

In [ ]:
for_genres_list_df.genres.unique()

In [ ]:
all_genres = list(for_genres_list_df.genres.unique())

In [ ]:
all_genres

In [ ]:
len(all_genres)

### Let's turn our genres column into a space separated list of genres (as if they were words in a document)

In [ ]:
movies_with_document_description_df = movies_unique_genres_df.copy()
movies_with_document_description_df['genres'] = movies_with_document_description_df['genres'].apply(lambda x: ' '.join(x))

In [ ]:
movies_with_document_description_df

In [ ]:
get_aladdin_example(movies_with_document_description_df)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(vocabulary=all_genres)
tfidf_matrix = tf.fit_transform(movies_with_document_description_df['genres'])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
cosine_sim

In [ ]:
cosine_sim.shape

In [ ]:
from analysis.utils.recommendation import get_similar_movies

similar_movies = get_similar_movies('Toy Story (1995)', cosine_sim, movies_with_document_description_df, 30)
similar_movies

### THAT'S IT! :)